In [39]:
# %load_ext autoreload
# %autoreload 2

import torch
import json
import pickle
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import numpy as np

In [40]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [47]:
#model_state_dict = torch.load("model_local//pytorch_model.bin", map_location='cpu') 
#model = BertForMaskedLM.from_pretrained('bert-base-uncased', state_dict=model_state_dict)
model = BertForMaskedLM.from_pretrained('model_local/test/model.tgz')

In [48]:
model.eval()
with open("dataset/dev-v2.0.json", 'r') as handle:
    jdata = json.load(handle)
    data = jdata['data']

In [49]:
def calc_prob_nn(context, question):
    gt_question = question
    gt_q_tokens = tokenizer.tokenize(gt_question)
    gt_indexed_q_tokens = tokenizer.convert_tokens_to_ids(gt_q_tokens)
    
    context_tokens = tokenizer.tokenize(context)
    indexed_context_tokens = tokenizer.convert_tokens_to_ids(context_tokens)
    
    tokens_tensor = torch.tensor([indexed_context_tokens + gt_indexed_q_tokens])
    segments_tensors = torch.tensor([0]*len(indexed_context_tokens) + [1]*len(gt_indexed_q_tokens))
    predictions = model(tokens_tensor, segments_tensors)
    
    total = 0
    context_len = len(context_tokens)
    q_len = len(gt_indexed_q_tokens)
    for i in range(q_len):
        preds = predictions[0, context_len+i]
        m = torch.nn.LogSoftmax(0)
        total += -m(preds)[gt_indexed_q_tokens[i]]
    entropy = total / q_len
    perplexity = torch.exp(entropy)
    return perplexity.item()


In [50]:
answerable_probs = []
unanswerable_probs = []
counter = 0
for i in range(1):#len(data)):
    section = data[i]['paragraphs']
    for sec in section:
        context = sec['context']
        qas = sec['qas']
        for j in range(1):#len(qas)):
            question = qas[j]['question']
            label = qas[j]['is_impossible']
            try:
                prob = calc_prob_nn(context, question)
            except:
                continue
            if label:
                unanswerable_probs.append(prob)
            else:
                answerable_probs.append(prob)
            counter += 1
            if counter % 100 == 0:
                print("Processed ", counter)


In [51]:
len(unanswerable_probs), len(answerable_probs)

(0, 39)

In [52]:
answerable_probs

[1.038330316543579,
 1.0055084228515625,
 1.0810725688934326,
 1.012487530708313,
 1.380039095878601,
 1.640554666519165,
 1.1055219173431396,
 1.0394196510314941,
 1.0550795793533325,
 1.0171822309494019,
 1.2519900798797607,
 1.336911916732788,
 1.0026198625564575,
 1.01677668094635,
 2.2120251655578613,
 1.0343737602233887,
 2.997744560241699,
 1.0117734670639038,
 1.0477514266967773,
 1.0544703006744385,
 1.3831052780151367,
 1.418177604675293,
 1.0180962085723877,
 1.1981474161148071,
 1.0166099071502686,
 1.3279578685760498,
 1.7596973180770874,
 1.0504934787750244,
 1.149644374847412,
 1.032496452331543,
 2.6909704208374023,
 1.014782428741455,
 1.3074921369552612,
 1.0248439311981201,
 1.0974032878875732,
 1.178715705871582,
 1.0841788053512573,
 1.1735104322433472,
 1.133049488067627]

In [ ]:
gt_question = question
gt_q_tokens = tokenizer.tokenize(gt_question)
gt_indexed_q_tokens = tokenizer.convert_tokens_to_ids(gt_q_tokens)

context_tokens = tokenizer.tokenize(context)
indexed_context_tokens = tokenizer.convert_tokens_to_ids(context_tokens)

tokens_tensor = torch.tensor([indexed_context_tokens + gt_indexed_q_tokens])
segments_tensors = torch.tensor([0]*len(indexed_context_tokens) + [1]*len(gt_indexed_q_tokens))
predictions = model(tokens_tensor, segments_tensors)

total = 0
context_len = len(context_tokens)
q_len = len(gt_indexed_q_tokens)
for i in range(q_len):
    preds = predictions[0, context_len+i]
    m = torch.nn.LogSoftmax(0)
    tmp =  -m(preds)[gt_indexed_q_tokens[i]]
    print ("max", max(-m(preds)))
    print ("min", min(-m(preds)))
    print (tmp)
    total += tmp
entropy = total / q_len
perplexity = torch.exp(entropy)
perplexity.item()
